# Postmortem Key-Item Extractor 

This notebook extracts root causes, action items, and owners from incident reports using Llama 3 via Ollama.

Make sure Ollama is running locally with the Llama 3 model pulled (`ollama pull llama3`).

## Step 1: Install Required Libraries

Install Python libraries for data handling and LLM integration.

In [13]:
!pip install transformers accelerate --quiet

import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

## Step 2: Load Incident Data

Read the incident report(s) from `raw_data.json`.

In [14]:
with open("raw_data.json", "r", encoding="utf-8") as f:
    incident_data = json.load(f)

# Optional: preview keys
list(incident_data.keys())

['affected_activity',
 'assignment_group',
 'assigned_to',
 'assistant_groups',
 'cie_assigned_to',
 'audience',
 'caller',
 'close_code',
 'close_notes',
 'contact_type',
 'crn_masks',
 'current_status_next_steps',
 'customer_facing_impact',
 'customers_impacted',
 'child_incidents',
 'detection_source',
 'long_description',
 'outage_start',
 'outage_end',
 'recurring_event',
 'severity',
 'short_description',
 'technical_bridge_start',
 'tribe',
 'problem',
 'caused_by_change_number',
 'caused_by_change',
 'detection_time',
 'identification_time',
 'mitigation_time',
 'was_customer_impacted',
 'impact_adjustment_factor',
 'unit_type',
 'units_affected',
 'total_units',
 'chronology',
 'disable_pager',
 'status',
 'state',
 'created',
 'updated',
 'href',
 'number',
 'linked_cases',
 'sys_id',
 'resolved',
 'resolved_by',
 'affected_ci_list',
 'created_by',
 'disruption_time',
 'comment_list']

## Step 3: Format the LLM Prompt

Create a prompt for Llama 3 to extract root cause, action items, and owners.

In [15]:
def format_prompt(incident: dict) -> str:
    context = json.dumps(incident, indent=2)
    prompt = f"""
You are a postmortem analyst. Extract the following from this incident report:
- Root Cause
- Action Items
- Responsible Owners

Respond in JSON format with keys: root_cause, action_items, owners.

Incident Report:
{context}
"""
    return prompt

## Step 4: Connect to Ollama and Generate Output

Send the prompt to Llama 3 via Ollama and collect the streamed response.

In [ ]:

import requests

def check_ollama():
    try:
        r = requests.get("http://localhost:11434")
        return r.status_code == 200
    except Exception:
        return False

def get_llm_output(prompt):
    ollama_url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3",  # Use smaller model for faster response
        "prompt": prompt,
        "options": {"num_predict": 512}
    }
    llm_output = ""
    try:
        with requests.post(ollama_url, json=payload, stream=True) as response:
            response.raise_for_status()
            for line in response.iter_lines():
                if line:
                    try:
                        part = json.loads(line.decode())
                        llm_output += part.get("response", "")
                    except Exception:
                        pass
    except Exception as e:
        print("Ollama connection or generation error:", e)
    return llm_output

if not check_ollama():
    print("Ollama server is not running. Please start Ollama and pull the llama3 model.")
else:
    prompt = format_prompt(incident_data)
    llm_output = get_llm_output(prompt)

## Step 5: Extract and Display JSON Output

Parse the LLM output to extract the structured JSON and display it.

In [26]:
# Improved JSON extraction and validation
def extract_json(text):
    start = text.find("{")
    end = text.rfind("}") + 1
    if start == -1 or end == -1:
        print("No JSON found in LLM output.")
        return {}
    try:
        return json.loads(text[start:end])
    except Exception as e:
        print("Error extracting JSON:", e)
        return {}

extracted_json = extract_json(llm_output)

# Display extracted JSON
import pprint
pprint.pprint(extracted_json)

# Display as DataFrame
df = pd.DataFrame([extracted_json])
df.head()

{'action_items': ['Cleaned up Fabcon DB in the SDN dev team in the dev '
                  'environment',
                  'Transferred cleaned-up DBs to respective smartnics along '
                  'with a fabcon snap restart'],
 'owners': ['@nikolay.parvov@bg.ibm.com',
            '@amul.sharma1@ibm.com',
            '@Lekshmanan S',
            '@Gopi.selvaraj',
            '@Aniruddh Prakash',
            '@John Haughton'],
 'root_cause': 'Corrupted security group ID within the fabcon DB structure '
               'preventing fabcon server from starting up correctly on DPUs on '
               'node mon3-qz1-sr3-rk168-s18.'}


,root_cause,action_items,owners
0,Corrupted security group ID within the fabcon ...,[Cleaned up Fabcon DB in the SDN dev team in t...,"[@nikolay.parvov@bg.ibm.com, @amul.sharma1@ibm..."


## Step 6: Save Results

Save the extracted key items to CSV and JSON files for further analysis.

In [27]:
df.to_csv("llama3_extracted_postmortem.csv", index=False)

with open("llama3_extracted_postmortem.json", "w") as f:
    json.dump(extracted_json, f, indent=2)